# Weighting by Mean-Variance Optimization

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import os

from utils import *

# Load Data

In [ ]:
# Load the first sheet of the provided Excel file
data_path = os.path.dirname(os.getcwd()) + '/data'
data_file = data_path + '/fpi_raw_data.xlsx'
df = pd.read_excel(data_file, sheet_name='Universe of broad assets', index_col=0, parse_dates=True)

# Set the 'Date' column as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [14]:
df

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.0152,0.0757,-0.0023,0.0009,0.0422,0.0050,0.0373,0.0021,-0.0025,-0.0084,-0.0318
2000-08-01,0.0099,-0.0491,0.0079,0.0063,-0.0115,0.0404,-0.0292,0.0107,0.0096,0.0005,0.0146
2000-08-02,0.0007,-0.0178,0.0015,0.0037,0.0098,-0.0095,-0.0005,0.0010,0.0112,0.0012,0.0187
2000-08-03,0.0192,0.0761,0.0072,0.0031,-0.0031,-0.0210,-0.0307,-0.0095,0.0061,-0.0262,0.0072
2000-08-04,0.0142,-0.0030,0.0066,0.0062,0.0164,-0.0249,0.0129,0.0042,-0.0012,-0.0036,0.0326
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.0203,0.0258,-0.0144,-0.0047,0.0135,0.0194,0.0043,0.0039,0.0070,-0.0145,-0.0239
2024-09-03,-0.0423,-0.0631,0.0225,0.0078,-0.0618,0.0019,-0.0384,-0.0399,0.0028,-0.0086,-0.0392
2024-09-04,-0.0033,-0.0040,0.0250,0.0097,-0.0039,0.0043,-0.0051,-0.0007,-0.0092,0.0020,-0.0186


In [15]:
# Function to calculate tangency weights with NNLS
def tangency_weights(returns, cov_mat=1, regularization=0.1):
    cov = returns.cov() * 52  # Annualize the covariance for weekly data
    if cov_mat != 1:
        cov_diag = np.diag(np.diag(cov))
        cov = cov_mat * cov + (1 - cov_mat) * cov_diag
    cov += np.eye(len(cov)) * regularization

    mu = returns.mean() * 52  # Annualize the returns
    y = np.ones((len(mu), 1))

    # Fit NNLS regression to get non-negative weights
    model = LinearRegression(fit_intercept=False, positive=True)
    model.fit(cov, y)
    weights = model.coef_.flatten()
    
    weights /= weights.sum()  # Scale weights to sum to 1
    
    tangency_wts = pd.Series(weights, index=returns.columns, name="Tangency Weights")
    return tangency_wts

# Function to calculate GMV weights with NNLS
def gmv_weights(tot_returns, regularization=0.1):
    cov = tot_returns.cov() * 52
    cov += np.eye(len(cov)) * regularization
    y = np.ones((len(cov), 1))

    model = LinearRegression(fit_intercept=False, positive=True)
    model.fit(cov, y)
    weights = model.coef_.flatten()
    
    weights /= weights.sum()
    
    gmv_wts = pd.Series(weights, index=tot_returns.columns, name="GMV Weights")
    return gmv_wts

In [16]:
# Function to calculate MV portfolio weights with NNLS
def mv_portfolio(target_ret, tot_returns, regularization=0.1):
    tangency_wts = tangency_weights(tot_returns, cov_mat=1, regularization=regularization)
    gmv_wts = gmv_weights(tot_returns, regularization=regularization)
    
    mu_tan = tot_returns.mean() @ tangency_wts
    mu_gmv = tot_returns.mean() @ gmv_wts
    
    # Check to avoid division by zero
    if mu_tan == mu_gmv:
        combined_weights = gmv_wts.values
    else:
        delta = (target_ret - mu_gmv) / (mu_tan - mu_gmv)
        combined_weights = delta * tangency_wts.values + (1 - delta) * gmv_wts.values

    model = LinearRegression(fit_intercept=False, positive=True)
    model.fit(np.eye(len(combined_weights)), combined_weights)
    final_weights = model.coef_.flatten()
    final_weights /= final_weights.sum()

    MV = pd.Series(final_weights, index=tot_returns.columns, name="MV Weights")
    return MV

In [27]:
# Calculate weights for each day using the past 4 weeks of data
target_return = 0.01
dailly_weights = []

for day in range(20, len(df)):
    look_back_data = df.iloc[day-20:day]  # Use the past 4 weeks
    weights = mv_portfolio(target_return, look_back_data, regularization=0.1)
    weights.name = df.index[day]  # Assign the date of the current day to the weights
    dailly_weights.append(weights)

# Combine all weekly weights into a DataFrame
daily_weights_df = pd.DataFrame(dailly_weights)

# Display the weekly weights
print("Daily Portfolio Weights (4-week look-back period):")
daily_weights_df

Daily Portfolio Weights (4-week look-back period):


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
2000-08-28,0.0837,0.0639,0.1009,0.1018,0.0767,0.0927,0.0832,0.0943,0.1025,0.0965,0.1038
2000-08-29,0.0829,0.0692,0.0999,0.1013,0.0801,0.0935,0.0864,0.0942,0.1009,0.0942,0.0974
2000-08-30,0.0823,0.0683,0.0996,0.1012,0.0816,0.0929,0.0831,0.0955,0.1030,0.0944,0.0980
2000-08-31,0.0837,0.0685,0.0998,0.1015,0.0815,0.0907,0.0833,0.0971,0.1036,0.0942,0.0962
2000-09-01,0.0828,0.0604,0.0984,0.1010,0.0820,0.0954,0.0850,0.0996,0.1040,0.0910,0.1006
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.0571,0.0347,0.1436,0.1476,0.0242,0.0895,0.0823,0.0565,0.1596,0.0884,0.1165
2024-09-03,0.0600,0.0387,0.1343,0.1426,0.0305,0.0872,0.0824,0.0572,0.1621,0.0862,0.1189
2024-09-04,0.0679,0.0361,0.1253,0.1344,0.0330,0.1113,0.0827,0.0650,0.1577,0.0904,0.0963
2024-09-05,0.0672,0.0356,0.1277,0.1352,0.0323,0.1093,0.0827,0.0648,0.1570,0.0913,0.0968


# Backtest on MVO weight result

In [30]:
df_aligned = df.reindex(daily_weights_df.index)

daily_portfolio_return = (df_aligned * daily_weights_df).sum(axis=1)

daily_portfolio_return_df = daily_portfolio_return.to_frame(name="MVO_return")
daily_portfolio_return_df

,MVO_return
2000-08-28,0.0038
2000-08-29,-0.0027
2000-08-30,0.0058
2000-08-31,0.0127
2000-09-01,0.0038
...,...
2024-08-30,-0.0010
2024-09-03,-0.0129
2024-09-04,0.0009
2024-09-05,0.0020


In [34]:
# Number of assets
num_assets = df.shape[1]

# Step 1: Calculate Equal-Weighted Return
equal_weights = np.full(num_assets, 1 / num_assets)  # Equal weight for each asset
equal_weighted_return = (df_aligned * equal_weights).sum(axis=1)

# Step 2: Calculate Risk-Parity Weights and Returns
# Assuming we approximate using inverse volatility for simplicity:
# Calculate the rolling 20-day standard deviation for each asset as a proxy for risk (volatility)
asset_volatility = df.rolling(window=20).std()

# Calculate inverse volatility weights
inv_vol_weights = 1 / asset_volatility
risk_parity_weights = inv_vol_weights.div(inv_vol_weights.sum(axis=1), axis=0)

# Calculate the risk-parity return
risk_parity_return = (df_aligned * risk_parity_weights).sum(axis=1)

# Step 3: Combine into a DataFrame
daily_portfolio_return_df['equal_weighted_return'] = equal_weighted_return
daily_portfolio_return_df['risk_parity_return'] = risk_parity_return

# Display the resulting DataFrame
daily_portfolio_return_df

,MVO_return,equal_weighted_return,risk_parity_return
2000-08-28,0.0038,0.0042,0.0004
2000-08-29,-0.0027,-0.0028,-0.0025
2000-08-30,0.0058,0.0058,0.0036
2000-08-31,0.0127,0.0139,0.0120
2000-09-01,0.0038,0.0048,0.0033
...,...,...,...
2024-08-30,-0.0010,0.0033,0.0011
2024-09-03,-0.0129,-0.0235,-0.0138
2024-09-04,0.0009,-0.0003,0.0008
2024-09-05,0.0020,0.0008,0.0010


In [35]:
calc_summary_statistics(daily_portfolio_return_df, annual_factor=252, correlations=False, keep_columns=['Annualized Mean',
                                                                                      'Annualized Vol',
                                                                                      'Min',
                                                                                      'Max',
                                                                                      'Skewness',
                                                                                      'Excess Kurtosis',
                                                                                      'Historical VaR',
                                                                                      'Historical CVaR',
                                                                                      'Max Drawdown',
                                                                                      'Peak',
                                                                                      'Bottom',
                                                                                      'Recover',
                                                                                      'Duration (days)']).T

Assuming excess returns were provided to calculate Sharpe. If returns were provided (steady of excess returns), the column "Sharpe" is actually "Mean/Volatility"


,MVO_return,equal_weighted_return,risk_parity_return
Annualized Mean,0.1082,0.1331,0.1211
Annualized Vol,0.1321,0.2424,0.1324
Min,-0.0496,-0.1433,-0.0602
Max,0.0448,0.1493,0.0590
Skewness,-0.3906,-0.2619,-0.2841
Excess Kurtosis,1.8149,11.5013,4.1131
Historical VaR (5.00%),-0.0135,-0.0216,-0.0130
Annualized Historical VaR (5.00%),-0.2150,-0.3433,-0.2064
Historical CVaR (5.00%),-0.0191,-0.0357,-0.0192
Annualized Historical CVaR (5.00%),-0.3033,-0.5671,-0.3042
